#### 경제 관련 낚시성 기사 탐지 모델
- 데이터 : json 파일
- 피처 : 뉴스 기사 제목, 본문
- 타겟 : 낚시성 기사 여부

In [1]:
### 모듈 로딩
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch import optim
import torch.optim.lr_scheduler as lr_scheduler

from konlpy.tag import *

from collections import Counter
from custom import *

import os
import re
import pickle

import pandas as pd
import json
import numpy as np

In [2]:
file_path = './data/'

In [3]:
label_list = os.listdir(file_path)
print(label_list)

['0', '1']


In [4]:
dict = {'text' : [] , 'label' : []}
for num in label_list:
    folder_path = file_path+num
    file_list = os.listdir(folder_path)
    for file in file_list:
        with open(folder_path+'/'+file, encoding='utf-8') as f:
            data = json.load(f)
            df = pd.json_normalize(data, record_path=["sourceDataInfo","sentenceInfo"])
            
            data = df['sentenceContent'].to_string()
            # 한글만 남기기 + 공백 여러개 -> 1개로 변환
            data = re.sub(r'[^ㄱ-힣\s]', '', data)
            data = re.sub(r'\s', ' ', data)

            dict['text'].append(data)
            dict['label'].append(int(num))

print(len(dict['text']), len(dict['label']))

41082 41082


In [5]:
textDF = pd.DataFrame(dict)
textDF.head()

,text,label
0,금융권이 최근 한국임팩트금융을 설립해 본격 활동을 시작한 데 이어 소외계층...,0
1,열린 마음으로 미래를 내다보고 인재를 중시하자 미래에셋의 경영...,0
2,포용적 금융 이란 저소득층 소상공인 농민 또한 이자 부담 완화 ...,0
3,전 세계 기업들이 해킹으로 몸살을 앓고 있는 가운데 일 커뮤니케이션즈네이트...,0
4,연세대 지속가능발전포럼의 세션이 일 연세대학교 백양누리홀...,0


In [6]:
print(textDF['text'][0])

     금융권이 최근 한국임팩트금융을 설립해 본격 활동을 시작한 데 이어 소외계층 지원에       한국 임팩트금융이 본격 활동을 시작한 데 이어 금융권이 금융소외 계층 지원사업에 속      지난 월 설립추진위를 발족했던 임팩트금융 추진위원회는 최근 유한회사 한국임팩트 금      민간주도로 설립된 한국임팩트 금융은 억원대 자금을 조성해 금융소외계층과 사회      한국임팩트금융 위원장으로 추대된 이헌재 전 경제부총리는 사회적 변화가 빠르고 문      지난 발족식에서 개인적으로 주택문제에 관심이 있다는 이 위원장의 발언으로 볼      주거 외에도 한국임팩트금융은 보육 보건 환경 등 한국 사회의 핵심 이슈를 해결하      그 밖에도 최종구 금융위원회 위원장은 지난 일장애인 금융 이용 제약 해소 방안                         고 밝히면서 그 움직임이 구체적으로 드러나고 있다      금융권이 문재인정부 출범 이후 강조되어 온 임팩트 금융의 첫 단추로 장애인 금     임팩트 금융은 사회적 약자를 포함한 모든 경제 주체가 저축 지급 결제 신용 보     이러한 분위기 속에서 금융권도 사회적 책임을 다하고 포용적 금융을 실천하기 위해 금     국민은행은 지난 월부터 사회적 배려가 필요한 장기 연체 채무자에 대한 금융 지     국민은행이 추진하는 금융 지원은 연체 기간이 오래 지난 특수 채권에 대한 채무 감면     특히 대상자 중 연대 보증으로 어려움을 겪는 사람들에 대해서는 최대 까지 감면     국민은행은 상환 의지가 있음에도 실업 불의의 사고 등으로 경제력을 상실해 채무 상                                 우리은행은 더 큰 금융을 내세웠다     이를 위해 포용적 생산적 신뢰의금융 등 개의 실무팀을 구성해 중금리 대출 확대     우리은행은 서민 금융 거점 점포를 개에서 개로 확대하고 서민 금융 지원 강화              장애인 노령자들을 위한 맞춤 서비스를 통해 편의성을 높이는 곳도 있다     신

In [7]:
# 훈련, 테스트용 데이터 나누기
train = textDF.sample(frac=0.9, random_state=10)
test = textDF.drop(train.index)

print(train.head(5).to_markdown())
print(len(train), len(test))

|       | text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [8]:
# 불용어 리스트 생성
stop_path = './stopwords.txt'

In [9]:
with open(stop_path, 'r', encoding='utf-8') as f:
    wordlist = f.readlines()
wordlist

['프\n',
 '뗴\n',
 '잠시\n',
 '채\n',
 '즉시\n',
 '드\n',
 '하도록시키다\n',
 '제\n',
 '하는바\n',
 '쓰\n',
 '으로써\n',
 '연이서\n',
 '삐\n',
 '결론을 낼 수 있다\n',
 '지만\n',
 '조용히\n',
 '보는데서\n',
 'ㅟ\n',
 'ㅘ\n',
 '탸\n',
 '깨\n',
 '똬\n',
 '여보세요\n',
 '잇따라\n',
 'ㅐ\n',
 '삼\n',
 '거바\n',
 '이 때문에\n',
 '소생\n',
 '쉿\n',
 '함께\n',
 '난\n',
 '정도에 이르다\n',
 '언제\n',
 '괴\n',
 '매번\n',
 '하는 김에\n',
 '거기\n',
 '초\n',
 '까\n',
 '바꾸어말하자면\n',
 '솨\n',
 '처\n',
 '꾸\n',
 '까닭으로\n',
 '그럼에도\n',
 '아이참\n',
 '곳\n',
 '더욱더\n',
 '그중에서\n',
 '흥\n',
 '양자\n',
 '차\n',
 'ㅈ\n',
 '라서\n',
 '나\n',
 '비로소\n',
 '쐬\n',
 '요만한걸\n',
 '이 외에\n',
 '허\n',
 '하는 것도\n',
 '겨\n',
 '절대\n',
 '지든지\n',
 '하물며\n',
 '요만한 것\n',
 '좌\n',
 '지말고\n',
 '이리하여\n',
 '좋아\n',
 '이었다\n',
 '하네요\n',
 '따위\n',
 '아하\n',
 '할때\n',
 '그렇지만\n',
 '두\n',
 '어떻게\n',
 '내일\n',
 '퐈\n',
 '료\n',
 '무엇때문에\n',
 '종\n',
 '게\n',
 '버\n',
 '하기에\n',
 '하기 보다는\n',
 '한켠으로는\n',
 '시각\n',
 '펴\n',
 '만은 아니다\n',
 '심지어\n',
 '설\n',
 '도달하다\n',
 '퍽\n',
 'ㅒ\n',
 '겹\n',
 '어기여차\n',
 '에도\n',
 '트\n',
 '타인\n',
 '나왔는데\n',
 '볘\n',
 '퇴\n',
 '세상에\n',
 '

In [10]:
stopwords = []
for word in wordlist:
    stopwords.append(word.replace('\n',''))
print(len(wordlist),len(stopwords))
print(stopwords[-10:])

1218 1218
['형', '다는', '이어', '이날', '없', '으며', '신', '주요', '열', '아서']


In [11]:
# 데이터 토큰화 및 단어사전 구축
komoran= Komoran()

In [12]:
train_tokens = [[token for token in komoran.morphs(sentence) if token not in stopwords] for sentence in train.text]
test_tokens =  [[token for token in komoran.morphs(sentence) if token not in stopwords] for sentence in test.text]

In [13]:
len(train_tokens), len(test_tokens)

(36974, 4108)

In [14]:
vocab = build_vocab(corpus=train_tokens, n_vocab=20000, special_tokens=['<PAD>', '<UNK>'])
token_to_id = {token:idx for idx, token in enumerate(vocab)}
id_to_token = {idx:token for idx, token in enumerate(vocab)}

print(vocab[:20])
print(len(vocab))

print(token_to_id)
print(id_to_token)

['<PAD>', '<UNK>', '원', '억', '금융', '통하', '기업', '사업', '지나', '시장', '대하', '지난해', '관계자', '지역', '올해', '성', '국내', '명', '고객', '대전']
20002
{'<PAD>': 0, '<UNK>': 1, '원': 2, '억': 3, '금융': 4, '통하': 5, '기업': 6, '사업': 7, '지나': 8, '시장': 9, '대하': 10, '지난해': 11, '관계자': 12, '지역': 13, '올해': 14, '성': 15, '국내': 16, '명': 17, '고객': 18, '대전': 19, '만원': 20, '대표': 21, '정부': 22, '대출': 23, '투자': 24, '상품': 25, '코로나': 26, '지원': 27, '서비스': 28, '가격': 29, '최근': 30, '대비': 31, '은행': 32, '기술': 33, '제품': 34, '경우': 35, '관련': 36, '금리': 37, '가능': 38, '증가': 39, '않': 40, '권': 41, '진행': 42, '이후': 43, '규모': 44, '산업': 45, '대상': 46, '개발': 47, '점': 48, '분기': 49, '업계': 50, '한국': 51, '현재': 52, '판매': 53, '상승': 54, '미국': 55, '브랜드': 56, '업체': 57, '기관': 58, '경제': 59, '제공': 60, '최대': 61, '운영': 62, '다양': 63, '회장': 64, '기존': 65, '매출': 66, '가장': 67, '소비자': 68, '주택': 69, '사회': 70, '확대': 71, '글로벌': 72, '관리': 73, '약': 74, '달': 75, '사용': 76, '경영': 77, '전년': 78, '서울': 79, '세계': 80, '생산': 81, '연구': 82, '상황': 83, '많': 84, '거래': 85, '환경': 86, '해외':

In [15]:
# 단어 사전 저장
with open('vocab.pkl', 'wb') as f:
    pickle.dump(token_to_id, f)

print("단어 사전이 저장되었습니다.")

단어 사전이 저장되었습니다.


In [16]:
# 정수 인코딩 및 패딩
unk_id = token_to_id['<UNK>']
train_ids = [[token_to_id.get(token, unk_id)for token in text] for text in train_tokens]
test_ids = [[token_to_id.get(token, unk_id)for token in text] for text in test_tokens]

max_length = 200
pad_id = token_to_id['<PAD>']
train_ids = pad_sequences(train_ids, max_length, pad_id)
test_ids = pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

[   59  4339     1  1316   766  2773     8 14800   371   311   426    36
 14379   717  6827  1490  1147  5158  1143  3272   665   311   426     9
  3086    30   766    36   797  1197   766  8417  1389     1   159  4245
     1   311   426  4731    24  4390   311   426  6423   300    83   876
     4   305  2690 15279   161  2182  4660   300   294   577     4   547
   311   426   426     4    85   155   311   426   311   426  6094   321
  1368   155   238    41   178    40   300     4   960   311   426   178
  5486   165   556   311   426    85  1003  1519  1917  1003  1664  1340
  3782 17591   311   426   178   308   960  4732   311   426    85  1290
  1340    52    22   178  1290  1340   771   420  1398  1529   121  5659
    65     4   178   771  1398  1529   209   420    10  1398  1529   748
  1344   794   535  5626   178  7018   399   311   426   771    44   241
  2684    85  1290  1340    35   963    24    55   174   358   311   426
  1368   420   421   178   161  2182  8839 10260   

In [17]:
# 데이터로더 적용
batch_size=32
train_ids = torch.tensor(train_ids)
test_ids = torch.tensor(test_ids)

train_labels = torch.tensor(train.label.values, dtype=torch.float32)
test_labels = torch.tensor(test.label.values, dtype=torch.float32)

train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [18]:
# 손실함수와 최적화함수 정의
n_vocab = len(token_to_id)
hidden_dim = 64
embedding_dim = 128
n_layers = 2

device = 'cuda' if torch.cuda.is_available() else 'cpu'

classifier = SentenceClassifier(n_vocab=n_vocab, hidden_dim=hidden_dim, 
                                embedding_dim=embedding_dim, n_layers=n_layers).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.Adam(classifier.parameters(), lr=0.001)
# 최적화 스케줄링 인스턴스 생성
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=8, verbose=True)

c:\Users\kdp\anaconda3\envs\TEXT_018_230_38\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [19]:
# 저장 경로
SAVE_PATH = './models/'
# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH) : os.makedirs(SAVE_PATH)   # 폴더 / 폴더 / ...  하위폴더까지 생성

In [20]:
epochs = 100
interval = 500
for epoch in range(epochs):
    print(f'[{epoch+1}/{epochs}]')
    train_model(classifier, train_loader, criterion, optimizer, device, interval)
    loss_val, score_val = test_model(classifier,test_loader, criterion, device)

    # 최적화 스케줄러 인스턴스 업데이트
    scheduler.step(score_val)
    print(f'scheduler.num_bad_epochs => {scheduler.num_bad_epochs}')

    SAVE_MODEL = SAVE_PATH + f'loss_{loss_val:.3f}_score{score_val:.3f}.pth'
    torch.save(classifier, SAVE_MODEL)

    if scheduler.num_bad_epochs >= scheduler.patience :
        print(f'성능 개선이 없어서 {scheduler.patience} EPOCH에 조기 종료함!')
        break

[1/100]
Train Loss 0 : 0.6970170736312866
Train Loss 500 : 0.6915267961705754
Train Loss 1000 : 0.68853935995302
Val Loss : 0.6793552064156347, Val Accuracy : 0.565238558909445
scheduler.num_bad_epochs => 0
[2/100]
Train Loss 0 : 0.6648733615875244
Train Loss 500 : 0.6786463694896051
Train Loss 1000 : 0.6761855043016828
Val Loss : 0.6560969433581182, Val Accuracy : 0.6131937682570594
scheduler.num_bad_epochs => 0
[3/100]
Train Loss 0 : 0.6326393485069275
Train Loss 500 : 0.6126663473075973
Train Loss 1000 : 0.5767346455917491
Val Loss : 0.5133480073869691, Val Accuracy : 0.7716650438169426
scheduler.num_bad_epochs => 0
[4/100]
Train Loss 0 : 0.45991286635398865
Train Loss 500 : 0.4586044839934675
Train Loss 1000 : 0.45697238425036646
Val Loss : 0.48696839162545613, Val Accuracy : 0.7850535540408958
scheduler.num_bad_epochs => 0
[5/100]
Train Loss 0 : 0.3949907422065735
Train Loss 500 : 0.4079212110318586
Train Loss 1000 : 0.4074600317499616
Val Loss : 0.4841546349862749, Val Accuracy :